# Deploy and run a Kubeflow Pipeline from outside the Kubeflow cluster: IAP version

This notebook shows how to deploy and run a Kubeflow Pipeline from outside the Kubeflow cluster.  The example requires a Google Cloud Platform (GCP) [project](https://cloud.google.com/free/), and an [IAP](https://cloud.google.com/iap/)-enabled cluster running on [Kubernetes Engine](https://cloud.google.com/kubernetes-engine/) (GKE).

The notebook walks through how to create a GCP service account with the necessary permissons, how to add the account as an 'IAP-secured Web App User', and how to create a Pipelines SDK client that can communicate with the cluster.

## Setup and configuration


### Deploy a Kubeflow cluster on GKE using IAP

Deploy an IAP-enabled Kubeflow cluster on GKE.  The [launcher web app](https://deploy.kubeflow.cloud/#/deploy) is recommended. Follow [the instructions](https://www.kubeflow.org/docs/started/getting-started-gke/#create-oauth-client-credentials) to create an Oauth client ID and secret. 

Note the client ID, which you'll need later in the notebook to create a Pipelines SDK client.


### Create a service account to use for the deployment

Next, create a service account to use, by visiting
[https://console.cloud.google.com/iam-admin/serviceaccounts](https://console.cloud.google.com/iam-admin/serviceaccounts).

![create a service account](https://storage.googleapis.com/amy-jo/images/kfp-deploy/Screenshot_2019-03-09_11_07_06.png)

Give the account project Editor permissions (or more restrictive configuration if desired), and also— this is required— **service account token creator** permissions.

![give the service account Editor permissions](https://storage.googleapis.com/amy-jo/images/kfp-deploy/Screenshot_2019-03-09_11_07_24.png)


![give the service account service account token creator permissions](https://storage.googleapis.com/amy-jo/images/kfp-deploy/Screenshot_2019-03-09_11_07_42.png)

If you would like to run this notebook locally, create and download a JSON key file for your service account.
Then, in your local environment, set the `GOOGLE_APPLICATION_CREDENTIALS` environment variable to point to the key file:

```
export GOOGLE_APPLICATION_CREDENTIALS=<your_key_file_path>
```


### Add the new service account as an IAP-secured web app user

Next, add the new service account as an IAP-secured Web App User.  Visit [https://console.cloud.google.com/security/iap](https://console.cloud.google.com/security/iap). 
Click on 'kubeflow/envoy', then click on **ADD MEMBER**

![](https://storage.googleapis.com/amy-jo/images/kfp-deploy/Screenshot_2019-03-09_11_29_14.png)

Add the new service account as an IAP-secured Web App User.

![](https://storage.googleapis.com/amy-jo/images/kfp-deploy/Screenshot_2019-03-09_11_30_24.png)


## Running the notebook example

### ML Engine Notebook

Create a ML Engine notebook instance, and then **configure it to use the new service account**.  This is a two-stage process: you'll first create the notebook VM, then modify the VM's associated service account. 

First visit [https://console.cloud.google.com/mlengine/notebooks/instances](https://console.cloud.google.com/mlengine/notebooks/instances) and create a **NEW INSTANCE** (or you can use an existing instance if you prefer).  Then, find the new instance in the list of GCE VMs and *update its service account information to use the new service account* as described [here](https://cloud.google.com/compute/docs/access/create-enable-service-accounts-for-instances#changeserviceaccountandscopes).

### Local Jupyter installation

To run the notebook locally, you'll need Jupyter and Python 3 installed.  Ensure that you've set the `GOOGLE_APPLICATION_CREDENTIALS` environment var to point to your service account credentials:

```
export GOOGLE_APPLICATION_CREDENTIALS=<your_key_file_path>
```


### Finally, the code

Now we're ready to run the example code.

First, install the Kubeflow Pipelines SDK. **You may need to restart your notebook kernel after you do the installation**.    
Make sure you're using Python 3. 
If you're running this notebook locally within a Conda environment, you may need to change `pip3` to `pip`.

In [ ]:
!pip3 install https://storage.googleapis.com/ml-pipeline/release/0.1.25/kfp.tar.gz --upgrade

Next, do some imports:

In [ ]:
import datetime

import kfp
import kfp.compiler as compiler
import kfp.dsl as dsl

from google.cloud import storage

Define a (very) simple example pipeline to run:

In [10]:
@dsl.pipeline(
  name='Sequential',
  description='A pipeline with two sequential steps.'
)
def sequential_pipeline(filename='gs://ml-pipeline-playground/shakespeare1.txt'):
  """A pipeline with two sequential steps."""

  op1 = dsl.ContainerOp(
     name='getfilename',
     image='library/bash:4.4.23',
     command=['sh', '-c'],
     arguments=['echo "%s" > /tmp/results.txt' % filename],
     file_outputs={'newfile': '/tmp/results.txt'})
  op2 = dsl.ContainerOp(
     name='echo',
     image='library/bash:4.4.23',
     command=['sh', '-c'],
     arguments=['echo "%s"' % op1.outputs['newfile']]
     )

Next we'll create an instance of the Kubeflow Pipelines client. Edit the following code to use your Kubeflow cluster's IAP endpoint, and the client ID for the oauth credential you created during IAP setup. You can find this ID, which should look like: `<id-string>.apps.googleusercontent.com`, displayed in the list of OAuth 2.0 client IDs on the APIs credentials page here: https://console.cloud.google.com/apis/credentials

In [ ]:
ts = int(datetime.datetime.utcnow().timestamp() * 100000)
client = kfp.Client(host='https://<your-deployment>.endpoints.<your-project>.cloud.goog/pipeline', 
                    client_id='<your-client-id>')


Compile the pipeline, and create a Pipelines `Experiment`:

In [ ]:
compiler.Compiler().compile(sequential_pipeline, '/tmp/sequential.tar.gz')
exp = client.create_experiment(name='sequential')

Finally, run the pipeline:

In [ ]:
res =  client.run_pipeline(exp.id, 'sequential_' + str(ts), 
                           '/tmp/sequential.tar.gz',
                          )
print(res)

------------------------------------
Copyright 2019, Google, LLC.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.